In [25]:
import pandas as pd
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json
from coordenadas import pasando_a_string
from coordenadas import get_coordenadas
from coordenadas import rellenando_datasetfsq
import foursquare
from config import dic_categorias
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [20]:
load_dotenv()

True

In [21]:
tokg = os.getenv("tokg")
tokf3 = os.getenv ("tokf3")
client2= MongoClient ("localhost:27017")

In [22]:
client = foursquare.Foursquare( access_token= tokf3 )
gmaps = googlemaps.Client(key= tokg )
db = client2.get_database("ciudades")

# he borrado todo lo que tenia de datasets, y empiezo directamente con lo que sería el flujo de la api

input: ciudad y pais

In [177]:
test_and_create ("buenos aires","argentina")

'se han introducido los campos'

In [171]:
test_and_create ("madrid", "españa")

'Ya está creada'

In [12]:
def test (c):
    lista= db.list_collection_names()
    for col in lista:
        s = list ( db[col].find({"coordenadas": c}) )
        if len (s) == 0:
                  pass
        else:
                  return False                 

In [173]:
def test_and_create( ciudad, pais):
    try: 
        busqueda= ciudad + ", " + pais #unimos en un string para pasarselo como búsqueda a gplaces
        coords=gmaps.geocode( address= busqueda)
        nombre = coords[0]['address_components'][0]['short_name'] # nos quedamos con el nombre, que será el de la colección
        a=coords[0]["geometry"]["location"]["lat"] 
        b=coords[0]["geometry"]["location"]["lng"]
        c = str (a) +"," + str(b) # Formato que luego entienda foursquare
        dic= {"nombre": nombre,
                "coordenadas": c,
                 "latitud":a,
                 "longitud":b}

            
        if test(c) == False:
            
            return "Ya está creada"
        
        else:
            
            pass
        
            db.create_collection (nombre)   
    except: "los datos introducidos no son correctos"
        
    return  get_coords (nombre, dic)

In [14]:
def get_coords(nombre, dic):
    collection= nombre 
    db[collection].insert_one (dic)
    coord= dic["coordenadas"]
    return get_data_fields (coord, collection)
        

In [176]:
def get_data_fields (coord, collection):
    tokf3 = os.getenv ("tokf3")
    client = foursquare.Foursquare( access_token= tokf3 )
    cantidad=[]
    dic_insert={}
    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        cantidad= len (lista)
        dice1={categoria: {"número":cantidad }}
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            try:
                tipo = lista[i]["categories"][0]["name"]
            except:
                pass
            try:
                direccion= lista[i]["location"]["formattedAddress"]
            except:
                pass
        
            dice2 = {"nombre": nombre, 
               "latitud":latitud,
               "longitud": longitud,
                "tipo" : tipo,
               "direccion":direccion }
            dice.append (dice2)          
        
            dice1[categoria].update ({"listado": dice})
        db[collection].insert_one (dice1)
    
    return "se han introducido los campos"

Crear una función que haga una query de X restaurantes ok
Crear un dataframe con esa query
Ese dataframe, pasandolo a geodataframe
Con ese dataftame, plotearlo en folium

Lo creo directamente para que sea un segundo imput del usuario, de hecho puede ser el principal, y si le da error, le saco el otro para que cree la ciudad. collection e item serán desplegables, y el número será la cantidad que quiera que salga. 

In [183]:
def query(collection, item, numero):
    # ver si se pueden poner varios items, modificar funcion caso
    a=list (db[collection]. find ({item: {"$exists": True}},{ item +".listado" :1, "_id":0}))
    try:
        listapeque= a[0]["restaurantes"]["listado"][:numero]
    except:
        return "No hay resultados que mostrar"
        
    df=pd.DataFrame (listapeque)
    
    return making_map (df,collection)

In [179]:
def making_map (df,collection):
    
    centro=a=list (db[collection]. find ({"coordenadas": {"$exists": True}},{"latitud":1,"longitud" :1, "nombre":1,"_id":0})) 
    
    lat=a[0]["latitud"]
    
    long=a[0]["longitud"]
    
    nombre= a[0]["nombre"]
    
    mapa = folium.Map(location= [lat, long], zoom_start= 15)
    
    return making_points (df, collection, mapa)
    
    

In [165]:
def making_points (df, collection, mapa):
    for i, row in df.iterrows():
        
        marcador = {
            "location":[row["latitud"], row["longitud"]],
            "tooltip" : row["nombre"]
        }
    
        if row["tipo"] == "Festivo":
            icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
        
        elif row["tipo"] == "Vispera":
            icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "black"
        )
        
        else:
            icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black"
            )
        Marker(**marcador,icon = icon ).add_to(mapa)
    return mapa

In [184]:
query ("muy muy", "restaurantes", 10)

'No hay resultados que mostrar'

In [180]:
query ("CABA", "restaurantes", 10)